In [184]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind

In [193]:
np.random.seed(13)
n = 1000

In [194]:
X_1 = np.random.binomial(1, 0.8, n)
X_2 = np.random.normal(0, 1, n)
X_3 = np.random.normal(0, 1, n)
X_4 = np.random.normal(0, 1, n)

In [195]:
D = np.random.binomial(1,0.5,n)

error = np.random.normal(0,1,n)

In [196]:
Y = 2*D + 0.5*X_1 - 0.3*X_2 + 0.2*X_3 + error

In [197]:
df = pd.DataFrame({
    "X1": X_1,
    "X2": X_2,
    "X3": X_3,
    "X4": X_4,
    "D": D,
    "Y": Y
})

df.head()

,X1,X2,X3,X4,D,Y
0,1,0.2304,0.6049,-1.1407,1,2.6561
1,1,0.4319,0.5119,0.1633,1,4.5747
2,0,1.1782,-1.9977,-0.0423,0,-0.9690
3,0,-0.2890,0.3562,-0.9004,0,-0.5324
4,0,-0.7727,-1.2997,-0.8798,0,-2.1326


In [198]:
vector_1 = ["X1", "X2", "X3", "X4"]

In [199]:
print("con t-test")

resultados_ttest = []
for var in vector_1:
    tratamiento = df[df["D"] == 1][var]
    control = df[df["D"] == 0][var]
    t_stat, p_val = ttest_ind(tratamiento, control)
    resultados_ttest.append({"Variable": var,
                    "Tratamiento": treated.mean(),
                    "Control": control.mean(),
                    "P value": p_val})


con t-test


In [200]:
tabla = pd.DataFrame(resultados)
print(tabla)

  Variable  Tratamiento  Control  P value
0       X1       0.0132   0.1352   0.4941
1       X2       0.0132  -0.0185   0.7364
2       X3       0.0132  -0.0675   0.3736
3       X4       0.0132  -0.0670   0.2230


In [163]:
print("con regresión")

import statsmodels.api as sm

resultados_regresion = []
for var in covariates:
    X = sm.add_constant(df['D'])
    y = df[var]
    model = sm.OLS(y, X).fit()
    resultados_regresion.append({
        "Variable": var,
        "Coeficiente": model.params['D'],
        "Error std.": model.bse['D'],
        "P value": model.pvalues['D']
    })


con regresión


In [164]:
tabla_2 = pd.DataFrame(resultados_regresion)
print(tabla_2)

  Variable  Coeficiente  Error std.  P value
0       X1       0.0166      0.0251   0.5083
1       X2       0.0097      0.0636   0.8792
2       X3      -0.0364      0.0636   0.5677
3       X4      -0.0468      0.0624   0.4539


In [165]:
import statsmodels.formula.api as smf

modelo_1 = smf.ols("Y ~ D", data=df).fit()
print(modelo_1.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.461
Model:                            OLS   Adj. R-squared:                  0.461
Method:                 Least Squares   F-statistic:                     853.9
Date:                Sun, 21 Sep 2025   Prob (F-statistic):          3.91e-136
Time:                        19:01:08   Log-Likelihood:                -1492.8
No. Observations:                1000   AIC:                             2990.
Df Residuals:                     998   BIC:                             2999.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4742      0.049      9.729      0.0

In [166]:
modelo_2 = smf.ols("Y ~ D + X1 + X2 + X3 + X4", data=df).fit()
print(modelo_2.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.539
Model:                            OLS   Adj. R-squared:                  0.537
Method:                 Least Squares   F-statistic:                     232.6
Date:                Sun, 21 Sep 2025   Prob (F-statistic):          2.00e-164
Time:                        19:01:09   Log-Likelihood:                -1414.5
No. Observations:                1000   AIC:                             2841.
Df Residuals:                     994   BIC:                             2870.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1215      0.078      1.562      0.1

In [167]:
print("Resultado sin controles:", modelo_1.params['D'])
print("Resultado con controles0:", modelo_2.params['D'])

Resultado sin controles: 1.9922215071605707
Resultado con controles0: 1.996584404420475


In [168]:
print("Error Estándar sin controles:", modelo_1.bse["D"])
print("Error Estándar con controles:", modelo_2.bse["D"])

Error Estándar sin controles: 0.06817561208633201
Error Estándar con controles: 0.06321753205129464


In [177]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [178]:
X_cov = df[["X1", "X2", "X3", "X4"]].values
y = df["Y"].values

In [179]:
lasso_cv = LassoCV(cv=10, random_state=13).fit(X_cov, y)

In [180]:
coeficientes = pd.Series(lasso_cv.coef_, index=["X1", "X2", "X3", "X4"])
print("Coeficientes estimados por LASSO:\n", coeficientes)

Coeficientes estimados por LASSO:
 X1    0.4349
X2   -0.2924
X3    0.1736
X4   -0.0000
dtype: float64


In [182]:
seleccionadas = coeficientes[coeficientes != 0].values.tolist()
print("\nCovariables seleccionadas en λmin:", seleccionadas)


Covariables seleccionadas en λmin: [0.43492315347324095, -0.2923939740465069, 0.17356667177108615]
